 ChIP-seq analysis

# align with bowtie2

In [4]:
# more details in this presentation 
# https://qcb.ucla.edu/collaboratory/workshops/w7-chip-seq-analysis/

In [5]:
# This workshop is for biologists to get familiar with codeing. 
# By no mean this will conver the most advanced development in the field, but I will share my experience, learning bioinformatics as a wet lab biologist. 
# And will mention some new trends in the field for future direction to look into. 

# I am also not native English speaker, be aware of typo issues and let me know. 
# I work at NIAID. this training is more of personal training. 
# Let me know if you have suggestions. 

# runing time of some of the computation can be long, we will not wait until if finishes. 
# I have intermediate cells, you can copy and from the shared folder. 
# There will be a month of time for you to try, after the workshop. 
# you can contact me at zhuy16@nih.gov.

In [6]:
pwd

/spin1/users/classes/NIEHS_NGS/NIEHS_NGS_Workshop/chip-seq


In [7]:
#ls

### sratoolkit already installed on biowulf

In [8]:
module load sratoolkit/2.9.6

[+] Loading sratoolkit 2.9.6  ... 


In [9]:
# fastq-dump -Z SRR502329 > data/STAT1_30m_IFNa.fastq

In [10]:
# fastq-dump -Z SRR502327 > data/STAT1_6h_IFNa.fastq
# fastq-dump -Z SRR502228 > data/INP_30m_IFNa.fastq
# fastq-dump -Z SRR502225 > data/INP_6h_IFNa.fastq

In [11]:
#rm STAT1_30m_IFNa.fastq

In [12]:
module load bowtie/2 samtools

[+] Loading bowtie  2-2.4.4 
[+] Loading samtools 1.12  ... 


### bowtie2 genome index on biowulf reference 

In [13]:
ls /fdb/igenomes/

00DOWNLOAD.log            Canis_familiaris         PhiX
00DOWNLOADS_20160309.log  Danio_rerio              Rattus_norvegicus
00DOWNLOAD.sh             Drosophila_melanogaster  Saccharomyces_cerevisiae
00DOWNLOAD.sh~            Gallus_gallus            Schizosaccharomyces_pombe
00DOWNLOAD.sh.200813      Homo_sapiens             Sus_scrofa
Arabidopsis_thaliana      Macaca_mulatta           temp
Bos_taurus                Mus_musculus             Zea_mays
Caenorhabditis_elegans    Pan_troglodytes


In [14]:
ls /fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/

genome.1.bt2  genome.3.bt2  genome.fa      genome.rev.1.bt2
genome.2.bt2  genome.4.bt2  genome.fa.fai  genome.rev.2.bt2


In [15]:
export BOWTIE2_INDEXES=/fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/

# the following step 2 & 3 takes a long time (24mins/each) to finish. Don't run them, I have runned them.

In [17]:
# optimized for biowulf
SECONDS=0
bowtie2  -x genome --threads=4  --no-unal --local -U fastq/STAT1_30m_IFNa.fastq -S 0r_STAT1_30m_IFNa.sam
# do some work
duration=$SECONDS
echo "$(($duration / 60)) minutes and $(($duration % 60)) seconds elapsed."

21192112 reads; of these:
  21192112 (100.00%) were unpaired; of these:
    10270948 (48.47%) aligned 0 times
    7482711 (35.31%) aligned exactly 1 time
    3438453 (16.23%) aligned >1 times
51.53% overall alignment rate
24 minutes and 55 seconds elapsed.


# batch processing

In [18]:
#mkdir 0r_script_align

In [19]:
#cd ..

In [20]:
#rm -rf 0r_script_align/*

In [45]:
cd fastq

### use a script to generate bash script for each sample.

In [46]:
#cat gen_script_align.sh:
#!/bin/sh
for i in *.fastq 
do
cat <<EOT >> ../0r_script_align/script_align_$(echo $i | sed 's/.fastq//g').sh
#!/bin/sh
module load samtools
module load bowtie/2
bowtie2  -x genome --threads=4  --no-unal --local -U fastq/$i -S sam/$(echo $i | sed 's/.fastq//g').sam
EOT
done

In [50]:
cd ..

### submit each script for alignment

In [51]:
for i in 0r_script_align/script_align_*.sh; do echo $i; done

0r_script_align/script_align_INP_30m_IFNa.sh
0r_script_align/script_align_INP_6h_IFNa.sh
0r_script_align/script_align_STAT1_30m_IFNa.sh
0r_script_align/script_align_STAT1_6h_IFNa.sh


In [52]:
for i in 0r_script_align/script_align_*.sh; do sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 $i; done

17070404
17070405
17070409
17070411


In [53]:
# same as this:
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_STAT1_30m_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_STAT1_6h_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_INP_30m_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_INP_6h_IFNa.sh 

# remove mitochondrial reads, unknown chr reads, convert sam to bam files.

In [21]:
module load samtools

[-] Unloading samtools 1.12  ... 
[+] Loading samtools 1.12  ... 


### convert sam files to bam files using samtools view
1. filter and remove alignments to mitochondria and uncharacterized locations.
2. convert sam to bam files.
3. count the total number of depth

In [1]:
cat bam.sh

sed '/chrM/d;/chrUn/d' $1.sam > $1_filtered.sam
samtools view -S -b $1_filtered.sam > $1.bam
samtools view -c $1.bam


In [ ]:
./bam.sh sam/STAT1_30m_IFNa
#10848585
./bam.sh sam/STAT1_6h_IFNa
#11100766           
./bam.sh sam/INP_30m_IFNa
#19147013
./bam.sh sam/INP_6h_IFNa
#19769206

In [8]:
#samtools view -c done_sam/INP_6h_IFNa.bam

19769206


In [ ]:
# normalization to library size

In [ ]:
samtools view -b -s 1.538 sam/INP_30m_IFNa.bam > sam/INP_30m_IFNa_11E6.bam
samtools view -b -s 1.564 sam/INP_6h_IFNa.bam > sam/INP_6h_IFNa_11E6.bam

In [24]:
samtools view --help

samtools view: unrecognised option '--help'

Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]

Options:
  -b       output BAM
  -C       output CRAM (requires -T)
  -1       use fast BAM compression (implies -b)
  -u       uncompressed BAM output (implies -b)
  -h       include header in SAM output
  -H       print SAM header only (no alignments)
  -c       print only the count of matching records
  -o FILE  output file name [stdout]
  -U FILE  output reads not selected by filters to FILE [null]
  -t FILE  FILE listing reference names and lengths (see long help) [null]
  -X       include customized index file
  -L FILE  only include reads overlapping this BED FILE [null]
  -r STR   only include reads in read group STR [null]
  -R FILE  only include reads with read group listed in FILE [null]
  -N FILE  only include reads with read name listed in FILE [null]
  -d STR1[:STR2]
           only include reads with tag STR1 and associated value STR2 [null]
           Th

: 1